In [113]:
import osmnx as ox
import networkx as nx
import sklearn
import numpy as np
import folium
from classes import person, visits, team
import in_out

In [34]:
# Populate tracked instances list

_class_list = (person.Patient, person.Clinician, visits.Visit, team.Team)

for cls in _class_list:
    cls.load_tracked_instances()

In [21]:
def display_route(coord_list):
    """
    Uses Folium to display a route given a list of coordinates
    :param coord_list: List of tuples containing coordinate information
    :param visit_list: Ordered list of visit objects to use for generating markers and the tooltip
    :return: Ordered list of appointment coordinates for optimal travel time
    """
    # Calculate central point to initialize map
    center_coord = coord_average(coord_list)

    # Initialize map and set boundaries.
    map = folium.Map(location=center_coord)
    map.fit_bounds(coord_list)

    color_list = list(folium.Icon.color_options)

    # Loop through each coord and create a new marker and tooltip
    for index, coord in enumerate(coord_list):
        tooltip = f"<center><h2>{index}. {coord}</h2></center>" \

        folium.Marker(
            coord,
            tooltip=tooltip,
            icon=folium.Icon(color=color_list[index], icon=f"circle-{index}")
        ).add_to(map)

    map.render()

def coord_average(coord_list):
    """
    Finds the average point between all coordinates in the coordinate list in order to center the map.
    :param coord_list: List of tuples containing coordinate information
    :return: Tuple of average longitude and latitude
    """
    # Split latitude and longitude
    lat_list = np.array([coord[0] for coord in coord_list])
    long_list = np.array([coord[1] for coord in coord_list])
    
    # Average latitude and longitude
    mean_lat = lat_list.mean()
    mean_long = long_list.mean()

    return mean_lat, mean_long

In [97]:
clin = in_out.load_obj(person.Clinician, "./data/Clinician/10000.pkl")
date = "20/12/2022"

clin.visits[date]

[10003, 10000, 10007, 10001, 10019, 10008]

In [98]:
visit_list = [in_out.load_obj(visits.Visit, f"./data/Visit/{visit_id}.pkl") for visit_id in clin.visits[date]]
coord_list = [visit.coord for visit in visit_list]
coord_list

[(47.6121162, -122.3430024),
 (47.5933101, -122.3322722),
 (47.6252232, -122.3154426),
 (47.7325024, -122.3548937),
 (47.6012584, -122.3296063),
 (47.602524, -122.3041695)]

In [99]:
coord_avr = coord_average(coord_list)
coord_avr

(47.62782238333333, -122.32989778333335)

In [100]:
def number_DivIcon(color,number):
    """ Create a 'numbered' icon
    
    """
    icon = folium.features.DivIcon(
            icon_size=(150,36),
            icon_anchor=(18,38),
            html="""<span class="fa-stack" style="font-size: 12pt">
                    <!-- The icon that will wrap the number -->
                    <span class="fa fa-circle-o fa-stack-2x" style="color:{:s}"></span>
                    <!-- a strong element with the custom content, in this case a number -->
                    <strong class="fa-stack-1x">{:02d}</strong>
                </span>""".format(color,number)
        )
    return icon

In [102]:
# Calculate central point to initialize map
center_coord = coord_average(coord_list)

# Initialize map and set boundaries.
map = folium.Map(location=center_coord)
map.fit_bounds(coord_list)

color_list = ('darkblue', 'purple', 'orange', 'lightblue', 'blue', 'pink', 'green', 'beige', 'lightgreen', 'lightred', 'red', 'lightgray')

# Loop through each coord and create a new marker and tooltip
for index, coord in enumerate(coord_list):
    tooltip = f"""
                <center><h4>{index+1}. {visit_list[index].patient_name}</h4></center>
                <p><b>Address</b>: {visit_list[index].address}</p>
                <p><b>Time Window</b>: {visit_list[index].time_earliest} - {visit_list[index].time_latest}</p>
                <p><b>Priority</b>: {visit_list[index].visit_priority}</p>
                <p><b>Complexity</b>: {visit_list[index].visit_complexity}</p>
                <p><b>Skills Required</b>: {visit_list[index].skill_list}</p>
                <p><b>Discipline Requested</b>: {visit_list[index].discipline}</p>
                """ 

    folium.Marker(
        coord,
        tooltip=tooltip,
        icon=folium.Icon(icon_color='white', color=color_list[index]),
    ).add_to(map)

    folium.Marker(
        coord,
        tooltip=tooltip,
        icon=number_DivIcon(color_list[index], index+1)
    ).add_to(map)

map

In [116]:
# Plot route on map

# Loop through each coord and create a new marker and tooltip
ox.settings.log_console=True
ox.settings.use_cache=True
# define the start and end locations in latlng
start_latlng = coord_list[0]
end_latlng = coord_list[1]
# location where you want to find your route
place     = 'Seattle, Washington, United States'
# find shortest route based on the mode of travel
mode      = 'drive'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'time'        # 'length','time'
# create graph from OSM within the boundaries of some 
# geocodable place(s)
graph = ox.graph_from_place(place, network_type = mode)
# find the nearest node to the start location
orig_node = ox.nearest_nodes(graph, start_latlng[0], start_latlng[1])
# find the nearest node to the end location
dest_node = ox.nearest_nodes(graph, end_latlng[0], end_latlng[1])
#  find the shortest path
shortest_route = nx.shortest_path(graph,
                                orig_node,
                                dest_node,
                                weight=optimizer)

ImportError: scikit-learn must be installed to search an unprojected graph

In [111]:
shortest_route_map = ox.plot_route_folium(graph, shortest_route)
shortest_route_map

NameError: name 'shortest_route' is not defined